In [3]:
# Import PyDrive and associated libraries
# This only needs to be done once per notebook
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
# This only needs to be done once per notebook
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [4]:
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz
file_id = '1M2S9M28Ruu0z5g_qEIuYYeOEidfC3Gox' # Check your own ID in GDrive
downloaded = drive.CreateFile({'id': file_id})

# Save file in Colab memory
downloaded.GetContentFile('araskargo.csv')

In [7]:
import pandas as pd
import re
import demoji
import string
import nltk
nltk.download('punkt')
from textblob import TextBlob

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [8]:
df = pd.read_csv('araskargo.csv')
df.sample(5)

,Unnamed: 0,Date,User,Tweet
3833,3833,2021-02-23 20:58:58+00:00,Fehmi60461853,#MasumlarApartmanı \nsorumsuz #araskargo suçlu...
9033,9033,2019-10-23 15:46:27+00:00,niluferustuner,@araskargo #araskargo kadar anlayışsız bir fir...
1836,1836,2021-09-13 08:18:41+00:00,cmuluduz,@araskargo @YurticiKargo @mngkargo @Surat_Dest...
5897,5897,2020-09-11 21:57:48+00:00,Esin44516542,Aras kargo yazmış ki alıcı kargoru kabul etmed...
4832,4832,2020-12-18 12:47:34+00:00,Fehmi60461853,@AntepliMamato Heeeey #araskargo! \nKargomun i...


In [9]:
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('turkish'))
print(stop_words)

{'eğer', 'belki', 'ki', 'ama', 'bu', 'o', 've', 'yani', 'sanki', 'neden', 'tüm', 'kim', 'mı', 'siz', 'mu', 'birkaç', 'ne', 'nerde', 'her', 'defa', 'veya', 'ise', 'daha', 'için', 'bazı', 'biz', 'diye', 'nerede', 'birşey', 'aslında', 'da', 'kez', 'ya', 'hepsi', 'hiç', 'çünkü', 'niçin', 'hem', 'ile', 'nereye', 'nasıl', 'hep', 'acaba', 'mü', 'az', 'biri', 'en', 'şey', 'niye', 'şu', 'de', 'gibi', 'çok'}


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [10]:
def cleanTweet(tweet):
    tweet = tweet.lower()
    tweet = demoji.replace(tweet,"")
    tweet = re.sub('\n',' ',tweet)
    tweet = re.sub(r'@\w+|\#','',tweet)
    tweet = tweet.translate(str.maketrans("","",string.punctuation))
    tweet = re.sub(r"http\s+|www\S+|https\S+","",tweet,flags=re.MULTILINE) # remove link 
    filtered_words = [word for word in TextBlob(tweet).words if word not in stop_words]
    return " ".join(filtered_words)

In [11]:
df['cleanTweet'] = df['Tweet'].map(cleanTweet)
df.sample(5)

,Unnamed: 0,Date,User,Tweet,cleanTweet
1834,1834,2021-09-13 09:36:14+00:00,saribon__,#araskargo çalışanlarınız tükenmez kalemle esa...,araskargo çalışanlarınız tükenmez kalemle esas...
5746,5746,2020-09-26 19:59:03+00:00,Fehmi60461853,Kargomun içi böyle attığınız için kırılmış ola...,kargomun içi böyle attığınız kırılmış olabilir...
10201,10201,2018-08-17 07:21:22+00:00,misskopat1,#araskargo 15.08.2018 tarihinden beri yanlış a...,araskargo 15082018 tarihinden beri yanlış adre...
5976,5976,2020-09-05 12:55:54+00:00,Bay_Temur,İstanbuldan Samsun’a kargo teslimatını en uzun...,i̇stanbuldan samsun ’ a kargo teslimatını uzun...
11241,11241,2017-03-21 09:43:45+00:00,EddaOrh,Kargom artık gelmeli #araskargo ..,kargom artık gelmeli araskargo


In [12]:
import gensim
from gensim.utils import simple_preprocess

In [13]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

In [14]:
data = df['cleanTweet'].to_list()
data_words = list(sent_to_words(data))

In [15]:
print(data_words[:1][0][:30])

['kargo', 'sirketlerinde', 'baskasının', 'kargosunu', 'alma', 'olayları', 'patladı', 'bana', 'sorarsanız', 'mecbur', 'kalmadıkca', 'kargo', 'degerli', 'esya', 'gondermeyin', 'soylemiyorlar', 'bana', 'sorarsanız', 'kargo', 'sistemleri', 'hacklendi', 'yurticikargo', 'araskargo', 'mngkargo', 'matura']


In [16]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

/usr/local/lib/python3.8/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [17]:
import gensim.corpora as corpora

# Create Dictionary
id2word = corpora.Dictionary(data_words)

# Create Corpus
texts = data_words

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1][0][:30])


[(0, 1), (1, 1), (2, 2), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 3), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 2), (19, 1), (20, 1)]


In [18]:
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=10, 
                                       random_state=100,
                                       chunksize=100,
                                       passes=10,
                                       per_word_topics=True)

In [19]:
from pprint import pprint
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]


[(0,
  '0.065*"araskargo" + 0.026*"kargo" + 0.024*"gittigidiyor" + '
  '0.020*"yurticikargo" + 0.016*"mngkargo" + 0.012*"rezil" + 0.012*"bir" + '
  '0.011*"geldi" + 0.010*"bi" + 0.008*"suratkargo"'),
 (1,
  '0.057*"araskargo" + 0.011*"bulamadık" + 0.011*"para" + 0.010*"bir" + '
  '0.010*"calısan" + 0.010*"kullanma" + 0.010*"kere" + 0.009*"sabah" + '
  '0.008*"hizmet" + 0.008*"cozum"'),
 (2,
  '0.072*"araskargo" + 0.055*"bir" + 0.017*"kargo" + 0.016*"yok" + '
  '0.014*"boyle" + 0.013*"musteri" + 0.013*"kadar" + 0.012*"kargom" + '
  '0.011*"ben" + 0.010*"hala"'),
 (3,
  '0.047*"araskargo" + 0.032*"dan" + 0.024*"hepsiburada" + 0.022*"indirim" + '
  '0.011*"buyuk" + 0.010*"telefonları" + 0.009*"varsa" + 0.008*"uzak" + '
  '0.007*"istanbula" + 0.007*"vermeyen"'),
 (4,
  '0.053*"araskargo" + 0.023*"sube" + 0.022*"pismanlıktır" + 0.015*"kargonun" '
  '+ 0.011*"kayyum" + 0.010*"haber" + 0.010*"allah" + 0.010*"kac" + '
  '0.009*"eticaret" + 0.008*"telefonlara"'),
 (5,
  '0.069*"borkhane" + 0.05

In [20]:
from gensim.models import CoherenceModel

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_words, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda)

Coherence Score:  0.41623721851689943


In [21]:
# supporting function
def compute_coherence_values(corpus, dictionary, k, a, b):
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=k, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=a,
                                           eta=b)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_words, dictionary=id2word, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

In [22]:
import numpy as np
import tqdm

grid = {}
grid['Validation_Set'] = {}

# Topics range
min_topics = 2
max_topics = 11
step_size = 1
topics_range = range(min_topics, max_topics, step_size)

# Alpha parameter
alpha = list(np.arange(0.01, 1, 0.3))
alpha.append('symmetric')
alpha.append('asymmetric')

# Beta parameter
beta = list(np.arange(0.01, 1, 0.3))
beta.append('symmetric')

# Validation sets
num_of_docs = len(corpus)
corpus_sets = [gensim.utils.ClippedCorpus(corpus, int(num_of_docs*0.75)), 
               corpus]

corpus_title = ['75% Corpus', '100% Corpus']

model_results = {'Validation_Set': [],
                 'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }

In [27]:
num_topics = 8

lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=num_topics, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=0.01,
                                           eta=0.9)

In [30]:
from sklearn.decomposition import LatentDirichletAllocation as LDA
from sklearn.feature_extraction.text import CountVectorizer

tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=10000)
tf = tf_vectorizer.fit_transform(data)
tf_feature_names = tf_vectorizer.get_feature_names_out()

In [31]:
lda = LDA(n_components=30, max_iter=5, learning_method='online', learning_offset=50.,random_state=0).fit(tf)

In [37]:
def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic %d:" % (topic_idx))
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))
display_topics(lda,tf_feature_names,20)

Topic 0:
cok dakika kitap şubesinden 15 yandı söylediler araba varış yapacağınız göndermeyin ulaşılmıyor yaparken etmiyorum merkeze yapalım tabii aradılar amk verdiyseniz
Topic 1:
ediyor devam işi bırakın kapıda genel alici bilgilendirme hakkında açıklama ödeme çalışmak bilgiler eğitim çalışma merkez gönderici pos şube çalışanlarını
Topic 2:
teşekkürler boşveren rketler ihtiyaç valla birdaha taşıyor nsan ücretsiz isi sende aktarmada samsun protestoediyorum ürünümüz yapamayız 265 koyun yeniyıl sıcak
Topic 3:
bir olmaz seni kadar bahar çiçekle destek tweet verdiğin atacağım azimle taşı deler sçn desen başlar tweetle derler bakalım yeneceğim
Topic 4:
taşırken almasaydın kargonun herkes adreste ver gidip sorumlusunu bedeli duysun istedim etmedin çi bulunamadın zararı kargonu lan isen kırıyor işte
Topic 5:
kargo beni evde allah pişmanlıktır aras şi dalga si versin bütün evden görür yoktunuz nternet ulaşılamıyor zi diyip geldik çözemeyince
Topic 6:
börkhane börk kalpak sezonürünleri haydi br

In [38]:
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
panel = pyLDAvis.sklearn.prepare(lda,tf, tf_vectorizer)

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.8/dist-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(
